# <center>Data Science Project Part 1:  Differential Privacy</center>
<center>DATA 558, Spring 2021</center>
<center>TAs: Alec Greaves-Tunnell and Ronak Mehta</center>


#### Name: Apoorv Sharma
#### Partner:  Hasnah Said
#### Summary of findings:

    A: M1 is not differentially private. There is no randomness involved when returning the results.

    B: 'scale' parm is not used for M3. Moreover, this method is also not differentially private.
    
    C: Implement Laplace Mechanism
    
    D: Check for differentialy privacy
    
    E: The data scaled on all of X, rather than just X_train
    
    F: The mechanism should only be trained using the train datasets, not the whole datatset. As a result, we also need to implement a custom `train_test_split` function so that we can also split the attributes in the correct manner. 
    
    G: Incorrect implementation of Bayes rule

## Dataset: Animals with Attributes v2

The data comes from Animals with Attributes v2 dataset, which contains images of 50 different types of animals, with each animal labeled with various attributes (whether it flies, whether it has a tail, etc.). In this example, we will not use the images, and treat the classes themselves as datapoints. That is, we will have $n = 50$ data points $(x_1, y_1), ..., (x_n, y_n)$, where $x_i \in \{0, 1\}^d$ is a binary vector of attributes, and $y_i \in \{0, 1\}$ is a binary label indicating whether the animal is an `ocean` animal or not. There were originally 85 attributes, but we have subsetted them to $d = 5$ features, namely:

- `horns` - whether the animal has horns.
- `tree` - whether the animal lives in a tree.
- `bulbous` - whether the animal is stocky.
- `fierce` - whether the animal is fierce.
- `arctic` - whether the animal lives in the arctic.

Additionally, we have one protected attribute, `flippers`, indicating whether the animal has flippers. This attribute is known for 49 of the animals, but is unknown for a held out animal, the `buffalo`. We will inspect in this notebook whether we can uncover the `buffalo`'s protected attribute using a machine learning model trained on the 49 points. If the model is privcy preserving, we should not be able to do this any better than if we did not have the model in hand. If not, then we will be significantly more confident by virtue of having access to the (outputs of the) model.

In [1]:
import numpy as np
import pickle

In [2]:
data = pickle.load(open("privacy_data.pkl", "rb"))

X = data['X'].to_numpy()
y = data['y'].to_numpy()
attr = data['z'].to_numpy()   # The attributes of the database entries.
x = data['animal'].to_numpy() # The targeted individual.

The analysis you are expected to critique begins below.

## <center>Differentially Private Logistic Regression for AwA2</center>

## Preprocessing

Standard preprocessing techniques are applied.

## Finding F

Implemented a custom `test_train_split` function so that we can split the attributes as well

In [3]:
def test_train_split_custom(X, y, attrs, test_size=0.2):
    
    # Shuffle X and y exactly the same!
    idx = np.random.permutation(X.shape[0])
    train_idx = int((1 - test_size) * len(X))
    
    training_idx, test_idx = idx[:train_idx], idx[train_idx:]
    
    # Split your dataset 
    X_train = X[training_idx,:]
    X_test = X[test_idx,:]
    
    y_train = y[training_idx]
    y_test = y[test_idx]
    
    attrs_train = attrs[training_idx]
    attrs_test = attrs[test_idx]
    
    return X_train, X_test, y_train, y_test, attrs_train, attrs_test

## Finding E

The data should be scaled on just the training data. This is so that we dont 'learn' anything from the test data

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

np.random.seed(10)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test, attrs_train, attrs_test = train_test_split(X, y, attr, test_size=0.2)
# X_train, X_test, y_train, y_test, attrs_train, attrs_test = test_train_split_custom(X, y, attr, test_size=0.2)

# scaler = StandardScaler().fit(X)
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print("X train shape:", X_train.shape)
print("y train shape:", y_train.shape)
print("X test shape:", X_test.shape)
print("y test shape:", y_test.shape)

X train shape: (39, 5)
y train shape: (39,)
X test shape: (10, 5)
y test shape: (10,)


## The Privacy Mechanism

Below, we will implement the mechanism that returns responses from the machine learning model given queries from the data analyst.

In [5]:
from abc import ABC, abstractmethod
from sklearn.linear_model import LogisticRegression

class Mechanism:
    
    @abstractmethod
    def __init__(self, database, **kwargs):
        pass
    
    @abstractmethod
    def respond(query):
        pass

First, we cover the machine learning model. We will use unregularied logistic regression to map the feature vector $x \in \{0, 1\}^5$ to its label $y \in \{0, 1\}$. The `query` from the data analyst can come in four forms.
- `all` - indicating that the mechanism should return responses (predicted labels) for all 49 training points in the database.
- `flippers` - indicating that the mechanism should return responses (predicted labels) for all training points in the database that have flippers.
- `no_flippers` - indicating that the mechanism should return responses (predicted labels) for all training points in the database that do not have flippers.
- `x` - a single feature vector to be predicted by the model, passed as a `numpy.ndarray`.

We implement three mechanisms, each of which will produce the response in different ways while preserving privacy.

In [6]:
class Mechanism1(Mechanism):
    
    def __init__(self, database, attr):
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.name = "Mechanism 1"
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            return self.model.predict(query.reshape(1, -1))[0]
        elif query == "all":
            return self.model.predict(self.X)
        elif query == "flippers":
            return self.model.predict(self.X[self.attr == 1])
        elif query == "no_flippers":
            return self.model.predict(self.X[self.attr == 0])
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")

class Mechanism2(Mechanism):
    
    def __init__(self, database, attr, prob=0.9):
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.name = "Mechanism 2"
        self.prob = prob
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            coin = np.random.binomial(1, self.prob)
            if coin == 1:
                return self.model.predict(query.reshape(1, -1))[0]
            else:
                return np.random.binomial(1, 0.5)
        elif query == "all":
            X = self.X
        elif query == "flippers":
            X = self.X[self.attr == 1]
        elif query == "no_flippers":
            X = self.X[self.attr == 0]
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")
            
        y = self.model.predict(X)
        coins = np.random.binomial(1, self.prob, size=y.shape)
        random_response = np.random.binomial(1, 0.5, size=y.shape)
        
        return y * coins + random_response * (1 - coins)

## Finding B:

There is a small bug, where the 'scale' parameter is not used. We comment this out from the class constructor. 

Moreover, M3 is not differentially private. Once the noise added to the data is figured out, the adversary can find out all the information. This can be by sampling from the mechanism thousands of times. A better alternative would be to use the Laplace distribution to add the noise. 

In [7]:
class Mechanism3(Mechanism):
    
    # def __init__(self, database, attr, scale=1):
    def __init__(self, database, attr):
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.name = "Mechanism 3"
        # self.scale = scale
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            noise = np.random.uniform(-0.1, 0.1, size=query.shape)
            return  self.model.predict((query + noise).reshape(1, -1))[0]
        elif query == "all":
            X = self.X
        elif query == "flippers":
            X = self.X[self.attr == 1]
        elif query == "no_flippers":
            X = self.X[self.attr == 0]
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")
            
        X = X + np.random.uniform(-0.1, 0.1, size=X.shape)
        return self.model.predict(X)

## Finding C

Here we implement the laplace mechanism to show how we can make mechanism 1 and 3 differentially private.

In [8]:
class LaplaceMechanism(Mechanism):
    
    def __init__(self, database, attr, scale=0.8):
        self.name = "Laplace"
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.scale = scale # Parameter for noise.
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            X = query + np.random.laplace(scale=self.scale, size=query.shape)
            return self.model.predict(np.array([X]).reshape(1, -1))[0]
        elif query == "all":
            X = self.X
        elif query == "flippers":
            X = self.X[self.attr == 1]
        elif query == "no_flippers":
            X = self.X[self.attr == 0]
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")
        
        X = X + np.random.laplace(scale=self.scale, size=X.shape)
        return self.model.predict(X)

## Finding F

We should only use the X_train and y_train to train the mechanisms. 

In [9]:
# database = (X, y)
database = (X_train, y_train)

m1 = Mechanism1(database, attrs_train)
m2 = Mechanism2(database, attrs_train)
m3 = Mechanism3(database, attrs_train)
m4 = LaplaceMechanism(database, attrs_train)

## Finding A

There is no randomness that is added into this mechanism. Each time we query m1 using 'no_flippers', we always receive a response that contains mostly 0's. This implies that all animals that do not have flippers, mostly do not live in the ocean. 

When we query the 'x' animal, we always get a prediction of 0; 'x' does not live in the ocean. 

As a result, we can predict, with a high certainty, that 'x' does not have flippers. Thus, this mechanism is not differentially private

In [10]:
label_no_flippers = m1.respond('no_flippers').mean()
label_flippers = m1.respond('flippers').mean()
label = m1.respond(x)

print(f'Flippers: {label_flippers}\nNo Flippers: {label_no_flippers}\n"x": {label}')

Flippers: 0.8333333333333334
No Flippers: 0.0
"x": 0


## The Advarsarial Attack

We justify our claim of privacy by showing that an attack fails. In other words, we want the probabilities surrounding whether the targetted animal (`buffalo`) having the attribute `flipper` to not change very much given the response $\hat{y}$from the mechanism. The prior probability (that is, prior to calling the mechanism) of having `flippers` is estimated using the number of animals in the training set that have flippers, which we assume is known by the attacker. We want:

$$
\mathbb{P}\left(\text{flippers}\mid \hat{y}(\text{buffalo}) =\text{ocean}\right) \approx \mathbb{P}\left(\text{flippers}\right)
$$

If we are much more confident about the value of this protected attribute (i.e. the probabilities are significantly higher or lower) after using the prediction for the `ocean` attribute of `buffalo` from the model, then the mechanism has failed to protect privacy.

In [11]:
# Use Bayes rule to get a value for the probability of `x` having attr == flippers.
np.random.seed(123)

prior = attrs_train.mean()
print("Prior: %0.3f" % prior)

for mech in [m1, m2, m3, m4]:
    label = mech.respond(x)

    prob_1_given_attr_1 = mech.respond("flippers").mean()
    prob_1_given_attr_0 = mech.respond("no_flippers").mean()

    if label == 1:
        prob_label_given_attr_1 = prob_1_given_attr_1
        prob_label_given_attr_0 = prob_1_given_attr_0
    else:
        prob_label_given_attr_1 = prob_1_given_attr_0
        prob_label_given_attr_0 = prob_1_given_attr_1
        
    posterior = prior * prob_label_given_attr_0 / (prior * prob_label_given_attr_1 + (1 - prior) *  prob_label_given_attr_0)

    print("%s Posterior: %0.3f" % (mech.name, posterior))

Prior: 0.154
Mechanism 1 Posterior: 0.182
Mechanism 2 Posterior: 0.180
Mechanism 3 Posterior: 0.182
Laplace Posterior: 0.179


Clearly, these probabilities have not changed very much from the prior. Thus, we can be assured that each mechanism preserves privacy.

## Finding G

Bayes rule has been incorrectly implemented in the cell block above. This incorrect implementation leads to the prior and posterior probabilities having a similar value. However, this is not the case. The correct implementation, below, shows that the prior and posterior probabilities are very different. 

In [12]:
# Use Bayes rule to get a value for the probability of `x` having attr == flippers.
np.random.seed(123)

prior = attrs_train.mean()
# prior = attr.mean()
print("Prior: %0.3f" % prior)

for mech in [m1, m2, m3, m4]:
    label = mech.respond(x)

    prob_1_given_attr_1 = mech.respond("flippers").mean()
    prob_1_given_attr_0 = mech.respond("no_flippers").mean()

    if label == 1:
        prob_label_given_attr_1 = prob_1_given_attr_1    #P(ocean|flippers)
        prob_label_given_attr_0 = prob_1_given_attr_0    #P(ocean|no flippers)
    else:
        prob_label_given_attr_1 = 1 - prob_1_given_attr_1   #P(not ocean|flippers)
        prob_label_given_attr_0 = 1 - prob_1_given_attr_0   #P(not ocean|no flippers)
        
    posterior = prior * prob_label_given_attr_1 / (prior * prob_label_given_attr_1 + (1 - prior) *  prob_label_given_attr_0)

    print("%s Posterior: %0.3f" % (mech.name, posterior))

Prior: 0.154
Mechanism 1 Posterior: 0.029
Mechanism 2 Posterior: 0.059
Mechanism 3 Posterior: 0.029
Laplace Posterior: 0.061


## Maintaining Accuracy

Trivially, we can always preserve privacy by injecting a sufficiently large amount of noise. While this may be good by one metric, we might completely destroy the predictive performance of the model! It is important to maintain a balance such that we still perform well on a test set, which we inspect below. Note that this step would normally be done on a validation set, and final performance would be computed on the test set.

In [13]:
from sklearn.metrics import accuracy_score

np.random.seed(123)

for mech in [m1, m2, m3, m4]:
    y_pred = np.array([mech.respond(x) for x in X_test])
    print("%s Test Accuracy: %0.2f" % (mech.name, accuracy_score(y_pred, y_test)))

Mechanism 1 Test Accuracy: 1.00
Mechanism 2 Test Accuracy: 0.90
Mechanism 3 Test Accuracy: 1.00
Laplace Test Accuracy: 0.90


## Finding D - Checking for Differential Privacy

In [14]:
def is_in_response_region(y):
    return y == 1

def check_differentially_private(epsilon, mechanism1, mechanism2, query, is_in_response_region, num_sims=1000):
    
    y1 = []
    y2 = []
    for i in range(num_sims):
        # Collect responses of each mechanism into 'y1' and 'y2'.
        y1.append(mechanism1.respond(query))
        y2.append(mechanism2.respond(query))
       
    # Compute probability that the responses are in the response region.
    prob1 = is_in_response_region(np.array(y1)).sum() / num_sims
    prob2 = is_in_response_region(np.array(y2)).sum() / num_sims
    
    # Check definition, and set to Boolean below.
    is_differentially_private = (prob1 <= np.exp(epsilon) * prob2)   
    
    return is_differentially_private

In [15]:
X2, y2 = X_train.copy(), y_train.copy()
X2[4, 3] = 1 - X[4, 3]

In [16]:
database = (X2, y2)

m12 = Mechanism1(database, attrs_train)
m22 = Mechanism2(database, attrs_train)
m32 = Mechanism3(database, attrs_train)
m42 = LaplaceMechanism(database, attrs_train)

In [17]:
epsilon = 0.001

for mech in [(m1, m12), (m2, m22), (m3, m32), (m4, m42)]:
    mech1, mech2 = mech
    is_diff_private = check_differentially_private(epsilon, mech1, mech2, x, is_in_response_region)
    
    print(f'{mech1.name}, {mech2.name}: Is Private?: {is_diff_private}')

Mechanism 1, Mechanism 1: Is Private?: True
Mechanism 2, Mechanism 2: Is Private?: True
Mechanism 3, Mechanism 3: Is Private?: True
Laplace, Laplace: Is Private?: True
